In [12]:
import boto3
import time
import pandas as pd

# Set the bucket name and document name based on your S3 URL
bucket_name = 'textract-console-us-east-2-473b0b2a-4bbb-4f45-ae6e-997765bde30c'

# Set directory for output
folder = "/Users/beaubressler/Library/CloudStorage/Dropbox/Research/urban_assistance_program/"

# !! Set document name: This is the document you want to read in from AWS
document_name = 'project_directory_appendix_1966v2.pdf' 

# !! set output name: This is the name of the output csv you will produce
output_csv = folder + "data/digitization/intermediate/" + "project_directory_1966_appendix_a_textract.csv"



In [13]:
"""
This script uses Amazon Textract to analyze a document stored in an S3 bucket.
It retrieves tables from the document, handles large documents with asynchronous 
processing and pagination, and outputs the table data as a structured set of blocks 
for further analysis or DataFrame conversion.

### Workflow Steps:

1. **Setup and Start Document Analysis**:
    - Initialize Textract client and set the target S3 bucket and document name.
    - `client.start_document_analysis()` starts the document analysis job in Textract.
    - The job is launched asynchronously, meaning it runs in the background, which 
      is useful for large files. The job's ID is stored to track progress.

2. **Monitoring Job Status**:
    - The script enters a loop to repeatedly check the status of the job using
      `client.get_document_analysis()`.
    - Every 5 seconds, the script checks if the job is complete.
    - The loop exits once the job status is `"SUCCEEDED"`, allowing the script to 
      move to retrieving the results.

3. **Retrieving Paginated Results**:
    - Textract returns results in pages (pagination) when dealing with large files, 
      so a `next_token` is used to retrieve all pages one by one.
    - Each page is fetched using `client.get_document_analysis()`, with `next_token` 
      identifying where the next page starts.
    - The response data for each page is stored in a `blocks` list, which holds 
      the structure of tables and other data from the document.

4. **Filtering and Identifying Tables**:
    - Once all pages are retrieved, the script filters `blocks` to identify 
      only those blocks where `BlockType` is `"TABLE"`.
    - A count of the tables is printed for quick reference, and this data can be 
      further processed, converted to a DataFrame, or saved as a CSV as needed.

5. **Example Output**:
    - The blocks can then be processed and structured as needed, but this script 
      provides a foundation by fetching, identifying, and preparing the table data.

### Example Variables:
- `bucket_name`: S3 bucket where the document is stored.
- `document_name`: The document name in the S3 bucket to analyze.
- `folder` and `output_csv`: Local storage paths where results could be saved.

### Error Handling:
    - If the job status is not `"SUCCEEDED"`, an error message is printed to 
      indicate the job failed.

Example usage:
    Adjust `bucket_name`, `document_name`, and `folder` as needed, then run the 
    script to retrieve table data from the specified document.
"""

# Initialize Textract client
client = boto3.client('textract', region_name='us-east-2')

# Start document analysis (asynchronously for large files)
response = client.start_document_analysis(
    DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': document_name}},
    FeatureTypes=["TABLES"]
)

job_id = response['JobId']
print(f"Job started with ID: {job_id}")

# Check the status of the job
status = "IN_PROGRESS"
while status == "IN_PROGRESS":
    response = client.get_document_analysis(JobId=job_id)
    status = response['JobStatus']
    print(f"Job status: {status}")
    if status == "IN_PROGRESS":
        time.sleep(5)

# Retrieve paginated results once job is complete
if status == "SUCCEEDED":
    blocks = []
    next_token = None
    
    while True:
        # Fetch results with pagination
        if next_token:
            response = client.get_document_analysis(JobId=job_id, NextToken=next_token)
        else:
            response = client.get_document_analysis(JobId=job_id)
        
        # Add blocks to our list
        blocks.extend(response['Blocks'])
        
        # Check if there is another page of results
        next_token = response.get('NextToken')
        if not next_token:
            break  # Exit loop if no more pages

        print(f"Fetched page with {len(response['Blocks'])} blocks.")

    print(f"Total blocks retrieved: {len(blocks)}")

    # Process blocks to extract table data
    tables = [block for block in blocks if block['BlockType'] == 'TABLE']
    print(f"Extracted {len(tables)} table(s) from the document.")

    # (Optional) Process each table block and save to a DataFrame or CSV as needed

else:
    print(f"Job failed with status: {status}")

Job started with ID: 8b12c8e6e6994a129017144eeeb00d53cca5c49add0044ab75899a2c518840ce
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 blocks.
Fetched page with 1000 block

In [14]:
"""
This code takes JSON-formatted output from Amazon Textract, which organizes tables
as a series of blocks, and converts it into a structured pandas DataFrame. The steps are:
    
1. **Mapping Rows and Columns** (`get_rows_columns_map`):
    - This function organizes each table's data into rows and columns.
    - `table_result` holds raw table data, while `blocks_map` is a dictionary 
      for quick lookup of each block by its unique ID.
    - It finds each cell in the table by examining "child" relationships, retrieves 
      the row and column positions, and organizes cell content in a dictionary 
      using row and column indices.

2. **Extracting Text from Each Cell** (`get_text`):
    - Since each cell can contain multiple small blocks (like individual words),
      this function gathers and joins all text in each cell.
    - It loops through child blocks within a cell, collects each word, and joins them 
      into a complete sentence to produce the full text for each cell.

3. **Creating a DataFrame for Each Table** (`generate_table_df`):
    - Once rows and columns are mapped, this function converts them into a 
      structured pandas DataFrame.
    - It arranges each row in the correct column order, filling in any missing 
      cells with empty strings, so the table structure is maintained.
    - The output is a DataFrame with rows and columns correctly aligned.

4. **Processing All Tables and Saving as CSV** (`process_textract_tables`):
    - This main function manages the full document processing:
        - Maps all blocks by their IDs for efficient lookup.
        - Filters out blocks that represent tables only.
        - Converts each table into a DataFrame using the helper functions above, 
          combines all tables, and saves the final result as a CSV file.
    - The final output is a CSV file containing all tables from the document,
      formatted for easy analysis.

Example usage:
    Assuming `blocks` is the list of blocks returned from Textract:
    output_csv = "textract_output.csv"
    process_textract_tables(blocks, output_csv)
"""

import pandas as pd

# Function to map rows and columns for a single table
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map.get(child_id)
                if cell and cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        rows[row_index] = {}
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows

# Function to extract text from each cell block
def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map.get(child_id)
                    if word and word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
    return text.strip()

# Function to convert the extracted table data into a DataFrame
def generate_table_df(table_result, blocks_map):
    rows = get_rows_columns_map(table_result, blocks_map)
    table_data = []
    for row_index in sorted(rows.keys()):
        row = [rows[row_index].get(col, '') for col in sorted(rows[row_index].keys())]
        table_data.append(row)
    df = pd.DataFrame(table_data)
    return df

# Main processing function after pagination
def process_textract_tables(blocks, output_csv):
    # Map blocks for quick lookup
    blocks_map = {block['Id']: block for block in blocks}
    tables = [block for block in blocks if block['BlockType'] == 'TABLE']
    
    # Generate DataFrames for each table and concatenate
    table_dfs = [generate_table_df(table, blocks_map) for table in tables]
    final_df = pd.concat(table_dfs, ignore_index=True)

    # Print the final DataFrame and save to CSV
    print("Final DataFrame:\n", final_df)
    final_df.to_csv(output_csv, index=False)
    print(f"Data saved to {output_csv}")

# Example usage:
# Assuming `blocks` is the full list of blocks retrieved from Textract
# output_csv = "textract_output.csv"
# process_textract_tables(blocks, output_csv)

In [15]:
process_textract_tables(blocks, output_csv)

Final DataFrame:
                0                 1               2               3          4  \
0          Place            County  Project no(s).           Place     County   
1        ALABAMA                                    ALABAMA (Con.)              
2      Abbeville             Henry           P-3,4       Fairfield  Jefferson   
3     Adamsville         Jefferson           P-6,8        Fairhope    Baldwin   
4        Addison           Winston            P-10         Fayette    Fayette   
...          ...               ...             ...             ...        ...   
2559      Verona              Dane            P-20                              
2560       Waldo         Sheboygan            P-20                              
2561    Walworth          Walworth            P-15                              
2562    Waterloo         Jefferson            P-37                              
2563   Watertown  Dodge, Jefferson             P-1                              

         